# Tema: Limpeza de dados - Microdados Enem 


Esse trabalho utiliza os microdados do Exame Nacional do Ensino Médio (Enem)[1].

Fonte dos Dados INEP
> Link: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem



In [31]:
# Caminho das Funções no OS
import os 
import sys
sys.path.append(os.getcwd())

# Bancos de Dados
import sqlite3
import random

# Outros
from tqdm import tqdm #Barra de progresso
from math import sqrt

# Manipulação dos Dados
import pandas as pd
import numpy as np
from functions.data_manipulation import *

# set some options in pandas
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_info_columns', 200)
pd.set_option('display.max_info_rows', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 200)


In [3]:
# Abrindo o Banco de dados.
db_name = 'enem'
db = sqlite3.connect(f'{db_name}.db')

In [25]:
# Carregando os dados.

# 2022 a 2023 sep = ';'
# df_chunks = pd.read_csv('/home/manel/Project/Ciencia_de_Dados/Pos_Descomplica/Enem/dados/MICRODADOS_ENEM_2022.csv', encoding = "iso-8859-1", sep = ';', chunksize = 500000)
df_chunks = pd.read_csv('/home/manel/Project/Ciencia_de_Dados/Pos_Descomplica/Enem/dados/MICRODADOS_ENEM_2023.csv', encoding = "iso-8859-1", sep = ';', chunksize = 500000)

# 2015 sep = ','
#df_chunks = pd.read_csv('Enem\dados\MICRODADOS_ENEM_2015.csv', encoding = "iso-8859-1", sep = ',', chunksize = 500000)

for df in df_chunks:
    df.to_sql(
        name = 'enem_2023',    # Define o nome da tabela a receber as informações (ou ser criada) 
        index = False,         # Não escreve o índice do dataframe na tabela
        if_exists = 'append',  # Se a tabela já existir, adiciona as informações abaixo das existentes
        con = db               # Define em qual banco os dados serão inseridos
    )

In [26]:
pd.read_sql_query('select * from sqlite_master', db)

,type,name,tbl_name,rootpage,sql
0,table,enem_2022,enem_2022,2,"CREATE TABLE ""enem_2022"" (\n""NU_INSCRICAO"" INT..."
1,table,enem_2023,enem_2023,389776,"CREATE TABLE ""enem_2023"" (\n""NU_INSCRICAO"" INT..."


In [27]:
#Verificando tabelas 
query = """
    SELECT name 
    FROM sqlite_master
    WHERE type='table'
"""
name_tables = pd.read_sql_query(query, db)
name_tables

,name
0,enem_2022
1,enem_2023


In [37]:
pd.read_sql_query('select * from sqlite_master', db)

,type,name,tbl_name,rootpage,sql
0,table,enem_2022,enem_2022,2,"CREATE TABLE ""enem_2022"" (\n""NU_INSCRICAO"" INT..."
1,table,enem_2023,enem_2023,389776,"CREATE TABLE ""enem_2023"" (\n""NU_INSCRICAO"" INT..."


In [28]:
query = """
    SELECT * 
    FROM enem_2023
    LIMIT 5;
"""
enem_2023 = pd.read_sql_query(query, db)
enem_2023.head(2)
#enem_2019 = reduce_mem_usage(pd.read_sql_query(query,db))

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,5300108,Brasília,53,DF,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,F,E,D,5,F,C,C,D,C,D,C,B,B,D,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,5300108,Brasília,53,DF,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,E,E,B,3,H,A,B,C,C,A,B,B,B,A,B,A,B,B,A,A,C,A,D,B


In [29]:
enem_2022.shape

(5, 76)

In [33]:
# vamos ler apenas colunas específicas?
query = """
    SELECT NU_ANO, TP_SEXO, TP_ESTADO_CIVIL, TP_COR_RACA, TP_FAIXA_ETARIA,
           NO_MUNICIPIO_PROVA, SG_UF_PROVA,
           NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_LC, NU_NOTA_MT,
           NU_NOTA_REDACAO, IN_TREINEIRO,
           Q001, Q002, Q003, Q004, Q005, Q006, Q007, Q008,
           Q009, Q010, Q011, Q012, Q013, Q014, Q015, Q016, 
           Q017, Q018, Q019, Q020, Q021, Q022, Q023, Q024, Q025
    FROM enem_2022
"""

enem_2022 = reduce_mem_usage(pd.read_sql_query(query,db))

Memory usage of dataframe is 1007.78 MB


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:04<00:00,  9.02it/s]

Memory usage after optimization is: 149.26 MB
Decreased by 85.2%


In [34]:
enem_2022.shape

(3476105, 38)

In [35]:
# vamos ler apenas colunas específicas?
query = """
    SELECT NU_ANO, TP_SEXO, TP_ESTADO_CIVIL, TP_COR_RACA, TP_FAIXA_ETARIA,
           NO_MUNICIPIO_PROVA, SG_UF_PROVA,
           NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_LC, NU_NOTA_MT,
           NU_NOTA_REDACAO, IN_TREINEIRO,
           Q001, Q002, Q003, Q004, Q005, Q006, Q007, Q008,
           Q009, Q010, Q011, Q012, Q013, Q014, Q015, Q016, 
           Q017, Q018, Q019, Q020, Q021, Q022, Q023, Q024, Q025
    FROM enem_2023
"""

enem_2023 = reduce_mem_usage(pd.read_sql_query(query,db))

Memory usage of dataframe is 1140.52 MB


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:05<00:00,  6.35it/s]

Memory usage after optimization is: 168.91 MB
Decreased by 85.2%


In [36]:
enem_2023.shape

(3933955, 38)

In [38]:
anos = [enem_2023, enem_2022] 

# Define uma lista das provas que serão analisadas
provas = ['MATEMATICA','CIENCIAS_NATUREZA', 'LINGUAGENS', 'HUMANAS','REDACAO']

print(enem_2023.shape)
print(enem_2022.shape)


#'CO_UF_PROVA':'COD_UF',

for ano in anos:
    # renomeia as colunas do data frame
    cols = {
        'NU_INSCRICAO':'Inscricao',
        'NU_ANO':'Ano',
        'TP_SEXO':'Genero',
        'TP_ESTADO_CIVIL':'Estado_Civil',
        'TP_COR_RACA':'Cor',
        'TP_FAIXA_ETARIA':'Faixa_Etaria',
        'NO_MUNICIPIO_PROVA':'Municipio',
        'SG_UF_PROVA':'SG_UF',
        'NU_NOTA_CN':'Ciencias_Natureza',
        'NU_NOTA_CH':'Humanas',
        'NU_NOTA_LC':'Linguagem',
        'NU_NOTA_MT':'Matemática',
        'NU_NOTA_REDACAO':'Redação',
        'IN_TREINEIRO':'Treineiro',
        'Q001': 'Escolaridade_pai',
        'Q002': 'Escolaridade_mae',
        'Q003': 'Ocupacao_pai',
        'Q004': 'Ocupacao_mae',
        'Q005': 'Pessoas_residencia',
        'Q006': 'Renda_mensal_familiar',
        'Q007': 'Empregada_domestica',
        'Q008': 'Quantidade_banheiros',
        'Q009': 'Quantidade_quartos',
        'Q010': 'Quantidade_carros',
        'Q011': 'Quantidade_moto',
        'Q012': 'Geladeira',
        'Q013': 'Freezer',
        'Q014': 'Maquina_lavar',
        'Q015': 'Maquina_secar',
        'Q016': 'Microondas',
        'Q017': 'Lava_louca',
        'Q018': 'Aspirador_po',
        'Q019': 'TV',
        'Q020': 'DVD',
        'Q021': 'TV_assinatura',
        'Q022': 'Celular',
        'Q023': 'Telefone_fixo',
        'Q024': 'Computador',
        'Q025': 'Internet',
    }
    ano.rename(columns = cols, inplace = True)

(3933955, 38)
(3476105, 38)


In [41]:
enem_2023.head()

,Ano,Genero,Estado_Civel,Cor,Faixa_Etaria,Municipio,SG_UF,Ciencias_Natureza,Humanas,Linguagem,Matemática,Redação,Treineiro,Escolaridade_pai,Escolaridade_mae,Ocupacao_pai,Ocupacao_mae,Pessoas_residencia,Renda_mensal_familiar,Empregada_domestica,Quantidade_banheiros,Quantidade_quartos,Quantidade_carros,Quantidade_moto,Geladeira,Freezer,Maquina_lavar,Maquina_secar,Microondas,Lava_louca,Aspirador_po,TV,DVD,TV_assinatura,Celular,Telefone_fixo,Computador,Internet
0,2023,M,2,1,14,Brasília,DF,NaN,NaN,NaN,NaN,NaN,0,A,F,E,D,5,F,C,C,D,C,D,C,B,B,D,C,C,B,B,A,B,B,A,A,B
1,2023,M,2,1,12,Brasília,DF,NaN,NaN,NaN,NaN,NaN,0,F,E,E,B,3,H,A,B,C,C,A,B,B,B,A,B,A,B,B,A,A,C,A,D,B
2,2023,F,1,1,6,Caxias do Sul,RS,502.0,499.00,475.50,363.25,700.0,0,H,E,C,F,5,C,A,B,D,B,A,B,A,B,A,B,A,A,B,A,A,A,A,A,B
3,2023,F,1,3,2,Fortaleza,CE,459.0,508.50,507.25,466.75,880.0,0,D,D,B,B,5,C,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,D,A,A,B
4,2023,F,1,3,3,Quixadá,CE,402.5,379.25,447.00,338.25,560.0,0,B,B,A,A,4,B,A,B,A,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A


In [50]:
# Remove quem é treineiro.
# Não iremos analisar as notas dos alunos que tenham faltado algum dia de prova.
# Remove quem tirou nota 0 em alguma das provas

anos = [enem_2023, enem_2022] 
provas = ['Matemática','Ciencias_Natureza', 'Linguagem', 'Humanas','Redação']
for ano in anos:
    condicao = ano.loc[(ano['Treineiro']==1)].index
    ano.drop(condicao, inplace = True)
    ano.dropna(how = 'any', subset = provas, inplace = True)
    for prova in provas[:-1]:
        condicao = ano.loc[(ano[prova]==0)].index
        ano.drop(condicao, inplace = True)

In [44]:
print(enem_2023.shape)
print(enem_2022.shape)

(2152164, 38)
(1925479, 38)


In [45]:
enem_2023.head()

,Ano,Genero,Estado_Civel,Cor,Faixa_Etaria,Municipio,SG_UF,Ciencias_Natureza,Humanas,Linguagem,Matemática,Redação,Treineiro,Escolaridade_pai,Escolaridade_mae,Ocupacao_pai,Ocupacao_mae,Pessoas_residencia,Renda_mensal_familiar,Empregada_domestica,Quantidade_banheiros,Quantidade_quartos,Quantidade_carros,Quantidade_moto,Geladeira,Freezer,Maquina_lavar,Maquina_secar,Microondas,Lava_louca,Aspirador_po,TV,DVD,TV_assinatura,Celular,Telefone_fixo,Computador,Internet
2,2023,F,1,1,6,Caxias do Sul,RS,502.0,499.00,475.50,363.25,700.0,0,H,E,C,F,5,C,A,B,D,B,A,B,A,B,A,B,A,A,B,A,A,A,A,A,B
3,2023,F,1,3,2,Fortaleza,CE,459.0,508.50,507.25,466.75,880.0,0,D,D,B,B,5,C,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,D,A,A,B
4,2023,F,1,3,3,Quixadá,CE,402.5,379.25,447.00,338.25,560.0,0,B,B,A,A,4,B,A,B,A,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A
9,2023,M,1,1,11,Batatais,SP,564.5,630.50,610.50,680.00,600.0,0,H,E,F,D,2,F,A,B,C,B,B,B,B,B,A,B,A,B,C,B,A,C,A,B,B
10,2023,M,1,3,8,Natal,RN,645.0,620.00,627.00,736.50,860.0,0,F,C,D,B,4,B,A,C,C,A,A,B,A,B,A,B,A,B,B,A,A,E,A,B,B


In [46]:
df_23_22 = pd.concat([enem_2023, enem_2022])

In [49]:
print(enem_2022.shape)
df_23_22.head()

(1925479, 38)


,Ano,Genero,Estado_Civel,Cor,Faixa_Etaria,Municipio,SG_UF,Ciencias_Natureza,Humanas,Linguagem,Matemática,Redação,Treineiro,Escolaridade_pai,Escolaridade_mae,Ocupacao_pai,Ocupacao_mae,Pessoas_residencia,Renda_mensal_familiar,Empregada_domestica,Quantidade_banheiros,Quantidade_quartos,Quantidade_carros,Quantidade_moto,Geladeira,Freezer,Maquina_lavar,Maquina_secar,Microondas,Lava_louca,Aspirador_po,TV,DVD,TV_assinatura,Celular,Telefone_fixo,Computador,Internet
2,2023,F,1,1,6,Caxias do Sul,RS,502.0,499.00,475.50,363.25,700.0,0,H,E,C,F,5,C,A,B,D,B,A,B,A,B,A,B,A,A,B,A,A,A,A,A,B
3,2023,F,1,3,2,Fortaleza,CE,459.0,508.50,507.25,466.75,880.0,0,D,D,B,B,5,C,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,D,A,A,B
4,2023,F,1,3,3,Quixadá,CE,402.5,379.25,447.00,338.25,560.0,0,B,B,A,A,4,B,A,B,A,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A
9,2023,M,1,1,11,Batatais,SP,564.5,630.50,610.50,680.00,600.0,0,H,E,F,D,2,F,A,B,C,B,B,B,B,B,A,B,A,B,C,B,A,C,A,B,B
10,2023,M,1,3,8,Natal,RN,645.0,620.00,627.00,736.50,860.0,0,F,C,D,B,4,B,A,C,C,A,A,B,A,B,A,B,A,B,B,A,A,E,A,B,B


In [54]:
# Define coluna de médias
df_23_22['Media'] = df_23_22[provas].mean(axis = 1)

In [55]:
df_23_22.head()

,Ano,Genero,Estado_Civel,Cor,Faixa_Etaria,Municipio,SG_UF,Ciencias_Natureza,Humanas,Linguagem,Matemática,Redação,Treineiro,Escolaridade_pai,Escolaridade_mae,Ocupacao_pai,Ocupacao_mae,Pessoas_residencia,Renda_mensal_familiar,Empregada_domestica,Quantidade_banheiros,Quantidade_quartos,Quantidade_carros,Quantidade_moto,Geladeira,Freezer,Maquina_lavar,Maquina_secar,Microondas,Lava_louca,Aspirador_po,TV,DVD,TV_assinatura,Celular,Telefone_fixo,Computador,Internet,MEDIA,Media
2,2023,F,1,1,6,Caxias do Sul,RS,502.0,499.00,475.50,363.25,700.0,0,H,E,C,F,5,C,A,B,D,B,A,B,A,B,A,B,A,A,B,A,A,A,A,A,B,508.0,508.0
3,2023,F,1,3,2,Fortaleza,CE,459.0,508.50,507.25,466.75,880.0,0,D,D,B,B,5,C,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,D,A,A,B,564.5,564.5
4,2023,F,1,3,3,Quixadá,CE,402.5,379.25,447.00,338.25,560.0,0,B,B,A,A,4,B,A,B,A,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,425.5,425.5
9,2023,M,1,1,11,Batatais,SP,564.5,630.50,610.50,680.00,600.0,0,H,E,F,D,2,F,A,B,C,B,B,B,B,B,A,B,A,B,C,B,A,C,A,B,B,617.0,617.0
10,2023,M,1,3,8,Natal,RN,645.0,620.00,627.00,736.50,860.0,0,F,C,D,B,4,B,A,C,C,A,A,B,A,B,A,B,A,B,B,A,A,E,A,B,B,697.5,697.5


In [56]:
# Exportar para .csv. Sem o index, index=False; 
df_23_22.to_csv('df_23_22_new.csv', index=False, encoding = "utf-8")

In [ ]:
#Carregando os dados
# df_23_22 = pd.read_csv('df_23_22_new.csv', encoding = "utf-8", sep = ',')

In [57]:
df_23_22.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4077643 entries, 2 to 3476102
Data columns (total 40 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   Ano                    int16   
 1   Genero                 category
 2   Estado_Civel           int8    
 3   Cor                    int8    
 4   Faixa_Etaria           int8    
 5   Municipio              object  
 6   SG_UF                  category
 7   Ciencias_Natureza      float16 
 8   Humanas                float16 
 9   Linguagem              float16 
 10  Matemática             float16 
 11  Redação                float16 
 12  Treineiro              int8    
 13  Escolaridade_pai       category
 14  Escolaridade_mae       category
 15  Ocupacao_pai           category
 16  Ocupacao_mae           category
 17  Pessoas_residencia     int8    
 18  Renda_mensal_familiar  category
 19  Empregada_domestica    category
 20  Quantidade_banheiros   category
 21  Quantidade_quartos     category
 22 

In [59]:
# df_23_22.describe()

In [ ]:
fwefwgwgw